In [1]:
import glob
import json
import os

import boto3

from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain.document_loaders import (
    PDFPlumberLoader,
    PyPDFLoader,
)
from langchain.llms.bedrock import Bedrock, LLMInputOutputAdapter
from langchain.output_parsers import (
    PydanticOutputParser,
)
from langchain.prompts import PromptTemplate

from langchain.schema.document import Document

from utils import InnovationProfile, Readiness

VERBOSE = True

AZURE_OPENAI_API_BASE = "https://so-azure-openai-dev.openai.azure.com/"
AZURE_OPENAI_API_VERSION = "2023-07-01-preview"
AZURE_DEPLOYMENT_NAME = "gpt35-baseline"
AZURE_OPENAI_API_KEY = ""
AZURE_OPENAI_API_TYPE = "azure"

OPENAI_API_KEY = ""

session = boto3.Session(region_name='us-east-1')
boto3_bedrock = boto3.client(service_name='bedrock', region_name='us-east-1')
boto3_bedrock_stream = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

### Create Custom Bedrock class

In [2]:
from typing import Any, Iterator
from langchain.schema.output import GenerationChunk

class NewBedrock(Bedrock):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _prepare_input_and_invoke_stream(self, prompt, stop=None, run_manager=None, **kwargs) -> Iterator[GenerationChunk]:
        _model_kwargs = {"prompt": Any, "max_tokens_to_sample": 8192, "temperature": 0.0}  #  "max_tokens_to_sample": Any
        provider = "anthropic"
        # _model_kwargs[self.provider_stop_sequence_key_name_map.get(provider)] = stop
        model_kwargs = {**_model_kwargs, **kwargs}
        input_body = LLMInputOutputAdapter.prepare_input(provider, prompt, model_kwargs)
        body = json.dumps(input_body)
        try:
            response = boto3_bedrock_stream.invoke_model(
                body=body,
                modelId="anthropic.claude-v2",
                accept="application/json",
                contentType="application/json",
            )
        except Exception as e:
            raise ValueError(f"Error raised by bedrock service: {e}")

        for chunk in LLMInputOutputAdapter.prepare_output_stream(
            provider, response, stop
        ):
            yield chunk


    def _prepare_input_and_invoke(self, prompt, stop=None, run_manager=None, **kwargs):
        _model_kwargs = {"prompt": Any, "max_tokens_to_sample": 8192, "temperature": 0.2}  # "max_tokens_to_sample": Any
       # _model_kwargs = {"prompt": Any, "max_tokens_to_sample": 8192}  # "max_tokens_to_sample": Any
        provider = "anthropic"
        # _model_kwargs[self.provider_stop_sequence_key_name_map.get(provider)] = stop
        model_kwargs = {**_model_kwargs, **kwargs}
        input_body = LLMInputOutputAdapter.prepare_input(provider, prompt, model_kwargs)
        body = json.dumps(input_body)
        print(body)
        try:
            response = boto3_bedrock_stream.invoke_model(
                body=body,
                modelId="anthropic.claude-v2",
                accept="application/json",
                contentType="application/json",
            )
            text = LLMInputOutputAdapter.prepare_output(provider, response)

        except Exception as e:
            raise ValueError(f"Error raised by bedrock service: {e}")

        return text

### Load Evidence

In [3]:
root_path = "/home/ubuntu/data/2022"

_folder_paths = [f.path for f in os.scandir(root_path) if f.is_dir()]
result_codes = [os.path.basename(f) for f in _folder_paths]
ID_CODE_EVIDENCE_MAP = {r: [] for r in result_codes}

for p in _folder_paths:
    base = os.path.basename(p)
    files = [f for f in (glob.glob(os.path.join(p, '**', '*.xlsx'), recursive=True) + \
                glob.glob(os.path.join(p, '**', '*.pdf'), recursive=True) + \
                glob.glob(os.path.join(p, '**', '*.pptx'), recursive=True)) \
                if not f.endswith('result.pdf')]
    ID_CODE_EVIDENCE_MAP[base].extend(files)

print(ID_CODE_EVIDENCE_MAP)

{'2051': ['/home/ubuntu/data/2022/2051/report.pdf'], '856': [], '4071': ['/home/ubuntu/data/2022/4071/59f8c07c55a4552db8db85044cfebee6.pdf'], '3155': ['/home/ubuntu/data/2022/3155/1-s2.0-S147470652100125X-main.pdf'], '1124': ['/home/ubuntu/data/2022/1124/development-of-the-vietnamese-healthy-eating-index.pdf'], '2897': ['/home/ubuntu/data/2022/2897/20-12-22 Burundi pVAB adoption study report ABC_TAFS WCA.pdf'], '3341': ['/home/ubuntu/data/2022/3341/59f8c07c55a4552db8db85044cfebee6.pdf'], '1698': ['/home/ubuntu/data/2022/1698/TPGS_breeding_program.pdf'], '2182': ['/home/ubuntu/data/2022/2182/c9f1cceecd33f93443b172d09760c5e5.pdf'], '1283': ['/home/ubuntu/data/2022/1283/major_challenges_2022.pdf'], '2168': ['/home/ubuntu/data/2022/2168/HCJ4O8.pdf'], '2104': ['/home/ubuntu/data/2022/2104/H050804.pdf'], '3352': ['/home/ubuntu/data/2022/3352/247c5df1711838df9983c409afe9d715.pdf'], '2049': ['/home/ubuntu/data/2022/2049/lhac015.pdf'], '1881': ['/home/ubuntu/data/2022/1881/ar_manual_nov2017.pdf

### Load Results

In [4]:
root_path = "/home/ubuntu/data/2022"

_folder_paths = [f.path for f in os.scandir(root_path) if f.is_dir()]
result_codes = [os.path.basename(f) for f in _folder_paths]
ID_CODE_RESULT_MAP = {r: None for r in result_codes}

for code in result_codes:
    path = f"/home/ubuntu/data/2022/{code}/result.pdf"
    if os.path.exists(path):
        ID_CODE_RESULT_MAP[code] = path

print(ID_CODE_RESULT_MAP)

{'2051': '/home/ubuntu/data/2022/2051/result.pdf', '856': '/home/ubuntu/data/2022/856/result.pdf', '4071': '/home/ubuntu/data/2022/4071/result.pdf', '3155': '/home/ubuntu/data/2022/3155/result.pdf', '1124': '/home/ubuntu/data/2022/1124/result.pdf', '2897': '/home/ubuntu/data/2022/2897/result.pdf', '3341': '/home/ubuntu/data/2022/3341/result.pdf', '1698': '/home/ubuntu/data/2022/1698/result.pdf', '2182': '/home/ubuntu/data/2022/2182/result.pdf', '1283': '/home/ubuntu/data/2022/1283/result.pdf', '2168': '/home/ubuntu/data/2022/2168/result.pdf', '2104': '/home/ubuntu/data/2022/2104/result.pdf', '3352': '/home/ubuntu/data/2022/3352/result.pdf', '2049': '/home/ubuntu/data/2022/2049/result.pdf', '1881': '/home/ubuntu/data/2022/1881/result.pdf', '4075': '/home/ubuntu/data/2022/4075/result.pdf', '1112': '/home/ubuntu/data/2022/1112/result.pdf', '785': '/home/ubuntu/data/2022/785/result.pdf', '4141': '/home/ubuntu/data/2022/4141/result.pdf', '128': '/home/ubuntu/data/2022/128/result.pdf', '1032

### Load Azure OpenAI

In [5]:
llm_azure = AzureChatOpenAI(
    openai_api_base=AZURE_OPENAI_API_BASE,
    openai_api_version=AZURE_OPENAI_API_VERSION,
    deployment_name=AZURE_DEPLOYMENT_NAME,
    openai_api_key=AZURE_OPENAI_API_KEY,
    openai_api_type=AZURE_OPENAI_API_TYPE,
)

### Load OpenAI

In [6]:
llm_openai = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-4", temperature=0)

### Load Anthropic Claude 2

In [8]:
llm_bedrock = NewBedrock(
    model_id="anthropic.claude-v2",
)

In [9]:
result = llm_bedrock("What does a researcher at CGIAR do?")

{"prompt": "\n\nHuman: What does a researcher at CGIAR do?\n\nAssistant:", "max_tokens_to_sample": 8192, "temperature": 0.2}


In [10]:
print(result)

 CGIAR is a global research partnership that works to advance agricultural science and innovation for food security. Some examples of what researchers at CGIAR do include:

- Developing new crop varieties that are higher yielding, more nutritious, or more resistant to pests, diseases, and climate stresses. This can involve plant breeding, genetic engineering, etc.

- Studying sustainable ways to manage soils, water, forests, and fisheries. This aims to optimize agriculture while protecting the environment.

- Looking for ways to control pests and diseases that affect crops and livestock. This can mean new management strategies, natural predators, gene editing, etc.

- Analyzing policies and socioeconomic factors that impact smallholder farmers and food security. Researchers provide evidence to guide policy reforms. 

- Working with farmers directly to test and implement new technologies and practices in local contexts. This facilitates adoption and impact.

- Building capacity of natio

### Load Documents

#### Load PRMS Result, Pdf & Excel Evidence

In [ ]:
code = "1037"
evidence_list = ID_CODE_EVIDENCE_MAP.get(code)
result = PyPDFLoader(ID_CODE_RESULT_MAP.get(code)).load_and_split()

evidence_docs = []
# generate a list of *already loaded* P documents
for path in evidence_list:
    if os.path.splitext(os.path.basename(path))[1] == ".pdf":
        _evidence_docs = PyPDFLoader(path).load_and_split()
    elif os.path.splitext(os.path.basename(path))[1] == ".pptx":
        _evidence_docs = PDFPlumberLoader(path).load_and_split()
    else:
        fname = os.path.basename(path)
        print(f"Error: Unable to load {fname}")
        continue
    for _doc in _evidence_docs:
        evidence_docs.append(_doc)

print("Result: \n", result)
print("Evidence: \n", evidence_docs)

In [12]:
print(len(evidence_docs))

110


### Extract PRMS Results

In [13]:
output_parser = PydanticOutputParser(pydantic_object=InnovationProfile)
format_instructions = output_parser.get_format_instructions()

In [14]:
document_prompt = PromptTemplate(
            input_variables=["page_content"],
            template="{page_content}"
        )

In [15]:
extraction_template = (
    "You are a researcher at CGIAR. Your task is to review reports submitted by other researchers and evaluate "
    "how innovative they are across a number of dimensions. A researcher has submitted the following research results "
    "containing key points about the research they conducted. Extract the necessary information from the document provided "
    "and returning this information as a JSON instance.\n\n"

    "{format_instructions}\n\n"

    "The text you are extracting information from can be found below.\n\n"

    "{text}"
)
extraction_prompt = PromptTemplate.from_template(template=extraction_template, partial_variables={"format_instructions": format_instructions})

In [16]:
llm_chain = LLMChain(llm=llm_azure, prompt=extraction_prompt, verbose=VERBOSE)

In [17]:
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_prompt=document_prompt, document_variable_name="text", verbose=VERBOSE)

In [ ]:
output = stuff_chain.run({"input_documents": result})

In [19]:
print(output)

{
  "description": "The global rangelands data platform is being established in response to a gap in rangelands data identified by the UN Environment Assembly through Resolution L17 Innovations in Sustainable Rangelands and Pastoralism 2019 and the complimentary report: A Case of Benign Neglect. Knowledge Gaps About Sustainability in Pastoralism and Rangelands. With funding from the CGIAR Big Data Platform and through a consultative process the outline of the platform was conceptualised. A global rangelands atlas was launched as a first step. It is anticipated that the global rangelands data platform will be launched in 2023. It will bring together big data on rangelands, source data including exploring crowd-sourcing and be an interactive platform for producing reports and trends for multiple users including UN Agencies monitoring rangeland and development agencies and actors developing interventions for rangelands, researchers and academics, and ultimately pastoralists themselves.",


In [20]:
result = json.loads(output)

### Summarize Evidence

In [21]:
description = result["description"]
long_title = result["long_title"]
short_title = result["short_title"]
innovation_character = result["innovation_character"]
innovation_typology = result["innovation_typology"]
readiness_level = result["readiness_level"]
readiness_justif = result["readiness_justif"]

In [22]:
summary_template = (
     "Human:<admin>You are a researcher at CGIAR. Your task is to review projects submitted by other researchers and evaluate "
     "them across a number of dimensions.<admin>\n\nHuman: "
     "This task consist of the following steps:\n"
     "Step 1: Review the project title and its description. Stop and think about it. This will provide you a general understanding of the evidence that will be presented in the next step.\n"
     "Step 2: Review the evidence provided.\n"
     "Step 3: Review the table inside the XML tags <innovation_levels></innovation_levels> This table will be useful in carrying out the next step.\n"
     "Step 4: Generate a list of quotes from the evidence that highlight the important activities that took place as well as the findings made and any results. "
     "Identify if activites carried out are ideations, the development of basic principles, or the validation or testing of well-defined hypotheses. "
     "Also include quotes that characterize the setting for the activities carried out. "
     "Reflect on if the evidence contains well-defined hypotheses that are being validated or tested under fully-controlled conditions, semi-controlled conditions or uncontrolled conditions. "
     "It may be useful to reference the innovation levels (1 through 9) during this step."
     "Finally, make sure to take note of if this innovation is a technological innovation, a capacity development innovation or a policy/organizational/institutional innovation."
     "Write quotes down word for word inside <thinking></thinking> XML tags. This is a space for you to write down relevant content and will not be shown to the user.\n"
     "Step 5: Using the quotes inside <thinking></thinking>, write a 500 word summary in a professional, academic 3rd person voice. "
     "IMPORTANT: It is critical that you distinguish activities which have been carried out from activities which have been planned but not yet carried out."
     "Make sure to recount all major activities that took place within the evidence. "
     "In addition to all major activites, the summary should identify any potential innovations that might result from the activities noted and make sure to explicitly state them. "
     "Finally, close the summary by restating all key findings and next steps discussed.\n"
     "Step 6: Review the summary written in step 5. If necessary re-write it to emphasize conciseness without losing any details. Return the summary without any introduction between the XML tag <summary>.\n"
     "IMPORTANT: Keep in mind that the audience consist of academic researchers. Never refer to yourself in the first person in this summary. "
     "Be sure to read the entire set of instructions carefully before beginning. "
     "Do not go to the next step without making sure the previous step has been completed.\n"

     "<project_title>\n"
     "{short_title}\n"
     "</project_title>\n"

     "<description>\n"
     "{description}\n"
     "</description>\n"

     "<innovation_levels>\n"
     "<row>"
     "<level>0</level>"
     "<title>Idea</title>"
     "<definition>The innovation is in the idea stage. The innovation is not yet being implemented.</definition>"
     "</row>"
     "<row>"
     "<level>1</level>"
     "<title>Basic Research</title>"
     "<definition>The innovation's basic principles are being researched for their ability to achieve an impact.</definition>"
     "</row>"
     "<row>"
     "<level>2</level>"
     "<title>Formulation</title>"
     "<definition>The innovation's basic principles are being formulated or designed.</definition>"
     "</row>"
     "<row>"
     "<level>3</level>"
     "<title>Proof of Concept</title>"
     "<definition>The innovation's key concepts have been validated for their ability to achieve a specific impact.</definition>"
     "</row>"
     "<row>"
     "<level>4</level>"
     "<title>Controlled Testing</title>"
     "<definition>The innovation is being tested for its ability to achieve a specific impact under fully-controlled conditions.</definition>"
     "</row>"
     "<row>"
     "<level>5</level>"
     "<title>Model/Early Prototype</title>"
     "<definition>The innovation is validated for its ability to achieve a specific impact under fully-controlled conditions.</definition>"
     "</row>"
     "<row>"
     "<level>6</level>"
     "<title>Semi-controlled Testing</title>"
     "<definition>The innovation is being tested for its ability to achieve a specific impact under semi-controlled conditions.</definition>"
     "</row>"
     "<row>"
     "<level>7</level>"
     "<title>Prototype</title>"
     "<definition>The innovation is validated for its ability to achieve a specific impact under semi-controlled conditions.</definition>"
     "</row>"
     "<row>"
     "<field>8</level>"
     "<title>Uncontrolled Testing</title>"
     "<definition>The innovation is being tested for its ability to achieve a specific impact under uncontrolled conditions.</definition>"
     "</row>"
     "<row>"
     "<level>9</level>"
     "<title>Proven Innovation</title>"
     "<definition>The innovation is validated for its ability to achieve a specific impact under uncontrolled conditions.</definition>"
     "</row>\n"
     "<innovation_levels>\n"

     "<evidence>\n"
     "{text}\n"
     "</evidence>"
     "\n\nAssistant:\n<summary>\n"
)
summary_prompt = PromptTemplate.from_template(template=summary_template, partial_variables={"short_title": short_title, "description": description})

In [23]:
llm_chain = LLMChain(llm=llm_bedrock, prompt=summary_prompt, verbose=VERBOSE)

In [24]:
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_prompt=document_prompt, document_variable_name="text", verbose=VERBOSE)

In [ ]:
evidence_summary = stuff_chain.run({"input_documents": evidence_docs})
evidence_summary = "<summary>\n"+evidence_summary

In [30]:
print(evidence_summary)

<summary>
Here is a 500 word summary of the project in a professional, academic 3rd person voice:

The Global Rangelands Data Platform project aimed to establish the first online global data platform for monitoring rangelands and consolidating rangeland data. The project was led by the International Livestock Research Institute (ILRI) and GMV, and was awarded $100,000 by the CGIAR Platform for Big Data in Agriculture through the Inspire Challenge 2020. 

The project began by analyzing existing geospatial platforms to help inform the development of the Rangelands Data Platform. Meetings were held with FAO and GEOGLAM to learn about the architecture and design decisions behind their platforms, Earth Map and RAPP Map. This analysis indicated that a cloud-based solution using Google Earth Engine for storage and processing capabilities would be optimal for meeting user requirements.  

To gather stakeholder input, an online survey was conducted which received 77 responses. The results showe

### Evaluate Readiness Level

In [28]:
readiness_output_parser = PydanticOutputParser(pydantic_object=Readiness)
readiness_format_instructions = readiness_output_parser.get_format_instructions()

In [31]:
readiness_template = (
     "You are a researcher at CGIAR. Your task is to review projects submitted by other researchers and evaluate "
     "them across a number of dimensions. This task consist of the following steps:\n"
     "Step 1: Review the summary provided. It summarizes the work carried out as part of the project. Make sure to distinguish activities which were carried out from activities which were only planned.\n"
     "Step 2: Review the table inside the XML tags <innovation_levels></innovation_levels>. It contains a scale of innovation readiness that ranges from 1 to 9.\n"
     "Step 3: Use the innovation readiness scale to determine the cumulative readiness level of COMPLETED activities conducted as part of the project. IMPORTANT: No'planned, but not-yet-completed' activities should be considered when determining the readiness level.\n"
     "Step 4: In a professional, academic 3rd person voice, concisely justify why you selected this readiness level in at most 300 words.\n"
     "IMPORTANT: Keep in mind that the audience consist of academic researchers. Never refer to yourself in the first person in this summary. "
     "Refer to all actions in the past tense. Be sure to read the entire set of instructions carefully before beginning. "
     "Do not go to the next step without making sure the previous step has been completed.\n\n"

     "Innovation development refers to a new, improved, or adapted output or groups of outputs such as technologies, products and services, policies, "
     "and other organizational and institutional arrangements with high potential to contribute to positive impacts when used at scale. "
     "Innovations may be at early stages of readiness (ideation or basic research) or at more mature stages of readiness (delivery and scaling).\n\n"

     "<innovation_levels>\n"
     "<row>"
     "<level>0</level>"
     "<title>Idea</title>"
     "<definition>The innovation is in the idea stage. The innovation is not yet being implemented.</definition>"
     "</row>"
     "<row>"
     "<level>1</level>"
     "<title>Basic Research</title>"
     "<definition>The innovation's basic principles are being researched for their ability to achieve an impact.</definition>"
     "</row>"
     "<row>"
     "<level>2</level>"
     "<title>Formulation</title>"
     "<definition>The innovation's basic principles are being formulated or designed.</definition>"
     "</row>"
     "<row>"
     "<level>3</level>"
     "<title>Proof of Concept</title>"
     "<definition>The innovation's key concepts have been validated for their ability to achieve a specific impact.</definition>"
     "</row>"
     "<row>"
     "<level>4</level>"
     "<title>Controlled Testing</title>"
     "<definition>The innovation is being tested for its ability to achieve a specific impact under fully-controlled conditions.</definition>"
     "</row>"
     "<row>"
     "<level>5</level>"
     "<title>Model/Early Prototype</title>"
     "<definition>The innovation is validated for its ability to achieve a specific impact under fully-controlled conditions.</definition>"
     "</row>"
     "<row>"
     "<level>6</level>"
     "<title>Semi-controlled Testing</title>"
     "<definition>The innovation is being tested for its ability to achieve a specific impact under semi-controlled conditions.</definition>"
     "</row>"
     "<row>"
     "<level>7</level>"
     "<title>Prototype</title>"
     "<definition>The innovation is validated for its ability to achieve a specific impact under semi-controlled conditions.</definition>"
     "</row>"
     "<row>"
     "<field>8</level>"
     "<title>Uncontrolled Testing</title>"
     "<definition>The innovation is being tested for its ability to achieve a specific impact under uncontrolled conditions.</definition>"
     "</row>"
     "<row>"
     "<level>9</level>"
     "<title>Proven Innovation</title>"
     "<definition>The innovation is validated for its ability to achieve a specific impact under uncontrolled conditions.</definition>"
     "</row>\n"
     "<innovation_levels>\n"

     "<summary>\n"
     "{text}\n"
     "</summary>"

     "{format_instructions}"
     "Only return the resulting JSON object. DO NOT return any other text."
)
readiness_prompt = PromptTemplate.from_template(template=readiness_template, partial_variables={"format_instructions": readiness_format_instructions})

In [32]:
print(readiness_prompt)

input_variables=['text'] partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"readiness_level": {"title": "Readiness Level", "description": "The readiness level selected.", "type": "string"}, "readiness_level_summary": {"title": "Readiness Level Summary", "description": "The summary which justifies the readiness level selected.", "type": "string"}}, "required": ["readiness_level", "readiness_level_summary"]}\n```'} template="You are a researcher at CGIAR. Your task is to review projects submitted by other researchers and evaluate

In [33]:
llm_chain = LLMChain(llm=llm_openai, prompt=readiness_prompt, verbose=VERBOSE)

In [34]:
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_prompt=document_prompt, document_variable_name="text", verbose=VERBOSE)

In [35]:
summary = Document(page_content=evidence_summary, metadata={"source": "Claude v2"})

In [36]:
print(summary.page_content)

<summary>
Here is a 500 word summary of the project in a professional, academic 3rd person voice:

The Global Rangelands Data Platform project aimed to establish the first online global data platform for monitoring rangelands and consolidating rangeland data. The project was led by the International Livestock Research Institute (ILRI) and GMV, and was awarded $100,000 by the CGIAR Platform for Big Data in Agriculture through the Inspire Challenge 2020. 

The project began by analyzing existing geospatial platforms to help inform the development of the Rangelands Data Platform. Meetings were held with FAO and GEOGLAM to learn about the architecture and design decisions behind their platforms, Earth Map and RAPP Map. This analysis indicated that a cloud-based solution using Google Earth Engine for storage and processing capabilities would be optimal for meeting user requirements.  

To gather stakeholder input, an online survey was conducted which received 77 responses. The results showe

In [37]:
readiness_eval = stuff_chain.run({"input_documents": [summary]})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a researcher at CGIAR. Your task is to review projects submitted by other researchers and evaluate them across a number of dimensions. This task consist of the following steps:
Step 1: Review the summary provided. It summarizes the work carried out as part of the project. Make sure to distinguish activities which were carried out from activities which were only planned.
Step 2: Review the table inside the XML tags <innovation_levels></innovation_levels>. It contains a scale of innovation readiness that ranges from 1 to 9.
Step 3: Use the innovation readiness scale to determine the cumulative readiness level of COMPLETED activities conducted as part of the project. IMPORTANT: No'planned, but not-yet-completed' activities should be considered when determining the readiness level.
Step 4: In a professional, academic 3rd person voice, concisely justify why you selected this rea


> Finished chain.

> Finished chain.


In [38]:
print(readiness_eval)

{"readiness_level": "4", "readiness_level_summary": "The Global Rangelands Data Platform project has reached the 'Controlled Testing' stage of innovation readiness. The project has moved beyond the ideation, basic research, formulation, and proof of concept stages. The project team has analyzed existing geospatial platforms, conducted an online survey to gather stakeholder input, and launched a Rangelands Atlas as an initial step toward the full data platform. The structure for the Rangelands Data Platform has been outlined, including a Google Earth Engine backend for cloud storage and processing, and a Mapbox or open-source TerriaJS frontend for visualization. However, the full launch of the Rangelands Data Platform is pending secure funding for maintenance and upkeep. Therefore, the project has not yet moved into the 'Model/Early Prototype' stage, where the innovation would be validated for its ability to achieve a specific impact under fully-controlled conditions."}
